# Анализ сервиса вопросов и ответов по программированию

**Задача исследования**: С помощью SQL посчитать заданные метрики сервис-системы вопросов и ответов о программировании.

**Описание данных**: Данные для анализа содержатся в 6 таблицах.
1. `stackoverflow.badges` хранит информацию о значках, которые присуждаются за разные достижения:
    - `id` 	идентификатор значка, первичный ключ таблицы;
    - `name`	название значка;
    - `user_id`	  идентификатор пользователя, которому присвоили значок, внешний ключ, отсылающий к таблице `users`;
    - `creation_date`	дата присвоения значка.
    
    
2. `stackoverflow.post_types` содержит информацию о типе постов, их может быть два:
    - `id`	идентификатор поста, первичный ключ таблицы;
    - `type`  тип поста:
        - `Question` — пост с вопросом;
        - `Answer` — пост с ответом.
    
    
3. `stackoverflow.posts` содержит данные о постах:
    - `id`	идентификатор поста, первичный ключ таблицы;
    - `title`	заголовок поста;
    - `creation_date`	дата создания поста;
    - `favorites_count`	число, которое показывает, сколько раз пост добавили в «Закладки»;
    - `last_activity_date`	дата последнего действия в посте, например комментария;
    - `last_edit_date`	дата последнего изменения поста;
    - `user_id`	идентификатор пользователя, который создал пост, внешний ключ к таблице users;
    - `parent_id`	если пост написали в ответ на другую публикацию, в это поле попадёт идентификатор поста с вопросом;
    - `post_type_id`	идентификатор типа поста, внешний ключ к таблице post_types;
    - `score`	количество очков, которое набрал пост;
    - `views_count`	количество просмотров.
    
    
4. `stackoverflow.users` содержит информацию о пользователях:
    - `id`	идентификатор пользователя, первичный ключ таблицы;
    - `creation_date`	дата регистрации пользователя;
    - `display_name`	имя пользователя;
    - `last_access_date`	дата последнего входа;
    - `location`	местоположение;
    - `reputation`	очки репутации, которые получают за хорошие вопросы и полезные ответы;
    - `views`	число просмотров профиля пользователя.
    
    
5. `stackoverflow.vote_types` содержит информацию о типах голосов. Голос — это метка, которую пользователи ставят посту. Типов бывает несколько:
    - `id`	идентификатор типа голоса, первичный ключ;
    - `name`	название метки:
        - `UpMod` — такую отметку получают посты с вопросами или ответами, которые пользователи посчитали уместными и полезными.
        - `DownMod` — такую отметку получают посты, которые показались пользователям наименее полезными.
        - `Close` — такую метку ставят опытные пользователи сервиса, если заданный вопрос нужно доработать или он вообще не подходит для платформы.
        - `Offensive` — такую метку могут поставить, если пользователь ответил на вопрос в грубой и оскорбительной манере, например, указав на неопытность автора поста.
        - `Spam` — такую метку ставят в случае, если пост пользователя выглядит откровенной рекламой.
        
        
6. `stackoverflow.votes` содержит информацию о голосах за посты:
    - `id`	идентификатор голоса, первичный ключ;
    - `post_id`	идентификатор поста, внешний ключ к таблице posts;
    - `user_id`	идентификатор пользователя, который поставил посту голос, внешний ключ к таблице users;
    - `bounty_amount`	сумма вознаграждения, которое назначают, чтобы привлечь внимание к посту;
    - `vote_type_id`	идентификатор типа голоса, внешний ключ к таблице vote_types;
    - `creation_date`	дата назначения голоса.
    
    
    
    
**Ход исследования**

1. Проведем обзор данных.
2. Выполним 20 заданий и подведем итоги.  

## Обзор данных

Импортируем библиотеки:

In [1]:
import pandas as pd
import sqlalchemy as sa
pd.set_option('display.max_colwidth', False)

Устанавливаем параметры:

In [2]:
db_config = {
    'user': '', # имя пользователя
    'pwd': '', # пароль
    'host': '',
    'port': , # порт подключения
    'db': '' # название базы данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

#сохраняем коннектор:
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})

Чтобы выполнить SQL-запрос, напишем функцию с использованием pandas `get_sql_data`:

In [3]:
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
    '''Открываем соединение, 
    получаем данные из sql, 
    закрываем соединение'''
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)

Для вывода первых пяти строк и информации о таблицах напишем функцию `read_sql_tabl`

In [4]:
def read_sql_tabl(df):
    '''Функция для вывода 
    5 первых строк и общей
    информации о таблице
    Args:
        df: SQL запрос
    для вывода таблицы
    Returns:
        5 строк pd.DataFrame
        общая информация pd.DataFrame.
    '''    

    display(pd.io.sql.read_sql(sql=sa.text(df), con = engine).head())
    pd.io.sql.read_sql(sql=sa.text(df), con = engine).info()

### Обзор таблицы `stackoverflow.badges`
Используем функцию `read_sql_tabl` и выведем информацию о таблице `stackoverflow.badges`

In [5]:
read_sql_tabl('''SELECT * FROM stackoverflow.badges''')

,id,name,user_id,creation_date
0,82947,Teacher,994,2008-09-15 08:55:04
1,82949,Teacher,3893,2008-09-15 08:55:04
2,82950,Teacher,4591,2008-09-15 08:55:04
3,82951,Teacher,5196,2008-09-15 08:55:04
4,82952,Teacher,2635,2008-09-15 08:55:04


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330130 entries, 0 to 330129
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   id             330130 non-null  int64         
 1   name           330128 non-null  object        
 2   user_id        330130 non-null  int64         
 3   creation_date  330130 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 10.1+ MB


В таблице с данными о значках 4 столбца и 330130 строк. 2 столбца с числами, 1 с типом object, 1 содержит даты. Объем памяти 10.1 MB. Имеются пропуски в столбце *name*.

### Обзор таблицы `stackoverflow.post_types`

Выгрузим данные о типах постов.

In [6]:
read_sql_tabl('''SELECT * FROM stackoverflow.post_types''')

,id,type
0,1,Question
1,2,Answer


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      2 non-null      int64 
 1   type    2 non-null      object
dtypes: int64(1), object(1)
memory usage: 160.0+ bytes


В таблице с типами постов 2 столбца и 2 строки. 1 столбец содержит числа, а 2-й с типом object содержит текст. Объем памяти 160 bytes.

### Обзор таблицы `stackoverflow.posts`

Выгрузим данные о постах.

In [7]:
read_sql_tabl('''SELECT * FROM stackoverflow.posts''')

,id,title,creation_date,favorites_count,last_activity_date,last_edit_date,user_id,parent_id,post_type_id,score,views_count
0,4,Convert Decimal to Double?,2008-07-31 21:42:53,41,2018-07-02 17:55:27.247,2018-07-02 17:55:27,8,0,1,573,37080
1,6,Percentage width child element in absolutely positioned parent on Internet Explorer 7,2008-07-31 22:08:09,10,2016-03-19 06:10:52.170,2016-03-19 06:05:48,9,0,1,256,16306
2,7,None,2008-07-31 22:17:58,0,2017-12-16 05:06:57.613,2017-12-16 05:06:58,9,4,2,401,0
3,9,How do I calculate someone's age in C#?,2008-07-31 23:41:00,399,2018-07-25 11:57:14.110,2018-04-21 17:48:14,1,0,1,1743,480476
4,11,Calculate relative time in C#,2008-07-31 23:55:38,529,2018-07-05 04:00:56.633,2017-06-04 15:51:20,1,0,1,1348,136033


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243796 entries, 0 to 243795
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   id                  243796 non-null  int64         
 1   title               52345 non-null   object        
 2   creation_date       243796 non-null  datetime64[ns]
 3   favorites_count     243796 non-null  int64         
 4   last_activity_date  243796 non-null  object        
 5   last_edit_date      87139 non-null   datetime64[ns]
 6   user_id             243796 non-null  int64         
 7   parent_id           243796 non-null  int64         
 8   post_type_id        243796 non-null  int64         
 9   score               243796 non-null  int64         
 10  views_count         243796 non-null  int64         
dtypes: datetime64[ns](2), int64(7), object(2)
memory usage: 20.5+ MB


В таблице постов 11 столбцов и 243796 строк. 7 столбцов с числами, 2 с типом object, 2 содержат даты. Объем памяти 20.5 MB. Имеются пропуски в столбцах *title* и *last_edit_date*.

### Обзор таблицы `stackoverflow.users`

Выгрузим данные о пользователях.

In [8]:
read_sql_tabl('''SELECT * FROM stackoverflow.users''')

,id,creation_date,display_name,last_access_date,location,reputation,views
0,1,2008-07-31 14:22:31,Jeff Atwood,2018-08-29 02:34:23,"El Cerrito, CA",44300,408587
1,2,2008-07-31 14:22:31,Geoff Dalgas,2018-08-23 17:31:56,"Corvallis, OR",3491,23966
2,3,2008-07-31 14:22:31,Jarrod Dixon,2018-08-30 20:56:24,"Raleigh, NC, United States",13418,24396
3,4,2008-07-31 14:22:31,Joel Spolsky,2018-08-14 22:18:15,"New York, NY",28768,73755
4,5,2008-07-31 14:22:31,Jon Galloway,2018-08-29 16:48:36,"San Diego, CA",39172,11700


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24044 entries, 0 to 24043
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id                24044 non-null  int64         
 1   creation_date     24044 non-null  datetime64[ns]
 2   display_name      24042 non-null  object        
 3   last_access_date  24044 non-null  datetime64[ns]
 4   location          15599 non-null  object        
 5   reputation        24044 non-null  int64         
 6   views             24044 non-null  int64         
dtypes: datetime64[ns](2), int64(3), object(2)
memory usage: 1.3+ MB


В таблице с данными о пользователях 7 столбцов и 24044 строки. 3 столбца с числами, 2 с типом object, 2 содержит даты. Объем памяти 1.3 MB. Имеются пропуски в столбце *location*.

### Обзор таблицы `stackoverflow.vote_types`

Выгрузим данные о типах голосов.

In [9]:
read_sql_tabl('''SELECT * FROM stackoverflow.vote_types''')

,id,name
0,1,AcceptedByOriginator
1,2,UpMod
2,3,DownMod
3,4,Offensive
4,5,Favorite


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      15 non-null     int64 
 1   name    15 non-null     object
dtypes: int64(1), object(1)
memory usage: 368.0+ bytes


В таблице с типами голосов 2 столбца и 15 строк. 1 столбец содержит числа, а 2-й с типом object содержит текст. Объем памяти 368 bytes.

### Обзор таблицы `stackoverflow.votes`

Выгрузим данные о голосах за посты.

In [10]:
read_sql_tabl('''SELECT * FROM stackoverflow.votes''')

,id,post_id,user_id,bounty_amount,vote_type_id,creation_date
0,434364,146794,4918,NaN,5,2008-10-03
1,434366,135535,16511,NaN,3,2008-10-03
2,434372,162181,12818,NaN,1,2008-10-03
3,434380,21965,4002,NaN,1,2008-10-03
4,434401,165975,8161,NaN,2,2008-10-03


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58790 entries, 0 to 58789
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             58790 non-null  int64  
 1   post_id        58790 non-null  int64  
 2   user_id        58790 non-null  int64  
 3   bounty_amount  220 non-null    float64
 4   vote_type_id   58790 non-null  int64  
 5   creation_date  58790 non-null  object 
dtypes: float64(1), int64(4), object(1)
memory usage: 2.7+ MB


В таблице с данными о голосах за посты 6 столбцов и 58790 строк. 5 столбцов с числами, 1 с типом object, 2 содержит даты. Объем памяти 1.3 MB. Есть пропуски в столбце *bounty_amount*.

### Выводы

Для анализа предоставлено 6 таблиц:

1. `stackoverflow.badges`- В таблице с данными о значках 4 столбца и 330130 строк. Имеются пропуски в столбце *name*.
2. `stackoverflow.post_types` - в таблице с типами постов 2 столбца и 2 строки. 
3. `stackoverflow.posts` - В таблице постов 11 столбцов и 243796 строк. Имеются пропуски в столбцах *title* и *last_edit_date*. 
4. `stackoverflow.users` - В таблице с данными о пользователях 7 столбцов и 24044 строки. Имеются пропуски в столбце *location*.
5. `stackoverflow.vote_types` - В таблице с типами голосов 2 столбца и 15 строк.
6. `stackoverflow.votes` - В таблице с данными о голосах за посты 6 столбцов и 58790 строк. Есть пропуски в столбце *bounty_amount*.

## Выполнение заданий

### Найдем количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки»

Данные о постах содержатся в таблице `stackoverflow.posts`, а типы постов в таблице `stackoverflow.post_types`. 

In [11]:
get_sql_data(
'''
SELECT COUNT(p.id)
FROM stackoverflow.posts AS p
JOIN stackoverflow.post_types AS pt ON p.post_type_id = pt.id
WHERE type = 'Question'
    AND (score > 300
    OR favorites_count >= 100)
    
''')

,count
0,1355


Мы получили 1355 вопросов, которые набрали более 300 очков или не менее 100 раз были добавлены в "Закладки".

### Определим, сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлим до целого числа.

Для выполнения запроса используем эти же таблицы: `stackoverflow.posts` и `stackoverflow.post_types`. 

In [12]:
get_sql_data(
'''
WITH i AS
(SELECT 
        CAST(date_trunc('day', creation_date) AS date) AS dt,
        COUNT(user_id)
    FROM stackoverflow.posts AS p
    JOIN stackoverflow.post_types AS pt ON p.post_type_id = pt.id
    WHERE type = 'Question' 
        AND 
        CAST(date_trunc('day', creation_date) AS date) BETWEEN '2008-11-01' AND '2008-11-18'
    GROUP BY dt
    ORDER BY dt)  
    
SELECT ROUND(AVG(count))
FROM i
''')

,round
0,383.0


С 1 по 18 ноября 2008 года в день задавали по 383 вопроса, в среднем.

### Определим количество уникальных пользователей, получивших значки сразу в день регистрации. 

Данные о пользователях содержатся в таблице  `stackoverflow.users`, а о значках в таблице  `stackoverflow.badges`

In [13]:
get_sql_data(
'''
SELECT COUNT(DISTINCT b.user_id)
FROM stackoverflow.badges AS b
JOIN stackoverflow.users AS u ON b.user_id = u.id
WHERE b.creation_date::date = u.creation_date::date

''')

,count
0,7047


7047 уникальных пользователей получили значки сразу в день регистрации. Это 29.3% от общего количества пользователей.

### Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?

Для выполнения запроса нам нужны 3 таблицы:
- о пользователях;
- о постах;
- о голосах за посты.

In [14]:
get_sql_data(
'''
SELECT COUNT(DISTINCT p.id)
FROM stackoverflow.users AS u
JOIN stackoverflow.posts AS p ON u.id = p.user_id
JOIN stackoverflow.votes AS v ON p.id = v.post_id
WHERE u.display_name = 'Joel Coehoorn'

''')

,count
0,12


12 уникальных постов пользователя Joel Coehoorn получили голоса

### Выгрузим все поля таблицы vote_types. Добавим к таблице поле rank, в которое войдут номера записей в обратном порядке и отсортируем таблицу по полю id.

In [15]:
get_sql_data(
'''
SELECT *,
    RANK() OVER (ORDER BY id DESC)
FROM stackoverflow.vote_types
ORDER BY id

''')

,id,name,rank
0,1,AcceptedByOriginator,15
1,2,UpMod,14
2,3,DownMod,13
3,4,Offensive,12
4,5,Favorite,11
5,6,Close,10
6,7,Reopen,9
7,8,BountyStart,8
8,9,BountyClose,7
9,10,Deletion,6


Проранжировали записи в таблице `stackoverflow.vote_types` и отсортировали ее по возрастанию id

### Отберем 10 пользователей, которые поставили больше всего голосов типа Close. 
Отобразим таблицу из двух полей: идентификатор пользователя и количество голосов. Отсортируем данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

Для выполнения запроса используем таблицы о голосах за посты и типах голосов.

In [16]:
get_sql_data(
'''
SELECT v.user_id,
    COUNT(vt.name) AS count
FROM stackoverflow.votes AS v
JOIN stackoverflow.vote_types AS vt ON v.vote_type_id = vt.id
WHERE vt.name = 'Close'
GROUP BY v.user_id
ORDER BY count desc, v.user_id desc
LIMIT 10

''')

,user_id,count
0,20646,36
1,14728,36
2,27163,29
3,41158,24
4,24820,23
5,9345,23
6,3241,23
7,44330,20
8,38426,19
9,19074,19


Максимальное количество 36 голосов типа Close поставил пользователь с id 20646.

### Отберем 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.
Отобразим несколько полей:
- идентификатор пользователя;
- число значков;
- место в рейтинге — чем больше значков, тем выше рейтинг.

Пользователям, которые набрали одинаковое количество значков, присвоим одно и то же место в рейтинге.
Отсортируем записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [17]:
get_sql_data(
'''
WITH i as

    (SELECT user_id,
        COUNT(name) AS count
    FROM stackoverflow.badges
    WHERE creation_date::date between '2008-11-15' AND '2008-12-15'
    GROUP BY user_id
    ORDER BY count desc)
    
SELECT user_id,
    COUNT,
    DENSE_RANK() OVER (ORDER BY count desc)
FROM i
ORDER BY count desc, user_id
LIMIT 10

''')

,user_id,count,dense_rank
0,22656,149,1
1,34509,45,2
2,1288,40,3
3,5190,31,4
4,13913,30,5
5,893,28,6
6,10661,28,6
7,33213,25,7
8,12950,23,8
9,25222,20,9


Больше всех значков (149) за установленный период получил пользователь с id 22656.

### Определим, сколько в среднем очков получает пост каждого пользователя.
Сформируем таблицу из следующих полей:
- заголовок поста;
- идентификатор пользователя;
- число очков поста;
- среднее число очков пользователя за пост, округлённое до целого числа.

Не будем учитывать пост без заголовка, а также те, что набрали ноль очков.

In [18]:
get_sql_data(
'''
SELECT
    title,
    user_id,
    score,
    ROUND(AVG(score) OVER (PARTITION BY user_id)) AS avg_score
FROM stackoverflow.posts
WHERE title IS NOT NULL AND
       score != 0

''')

,title,user_id,score,avg_score
0,Diagnosing Deadlocks in SQL Server 2005,1,82,573.0
1,How do I calculate someone's age in C#?,1,1743,573.0
2,Why doesn't IE7 copy <pre><code> blocks to the clipboard correctly?,1,37,573.0
3,Calculate relative time in C#,1,1348,573.0
4,Wrapping StopWatch timing with a delegate or lambda?,1,92,573.0
...,...,...,...,...
47349,Multi-lingual projects in Visual Studio,3742716,1,1.0
47350,Change datatype when importing Excel file into Access,3930756,5,5.0
47351,How to host a Silverlight app in a Sharepoint 2007 Web Part,4020932,8,8.0
47352,Getting counts for a paged SQL search stored procedure,5696608,2,2.0


В столбце *avg_score* содержится среднее количество очков у пользователя за посты

### Отобразим заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков исключим из списка.
Для выполнения запроса используем таблицы с информацией о пользователях и о значках.

In [19]:
get_sql_data(
'''
SELECT title
FROM stackoverflow.posts AS p
WHERE user_id IN 
   (SELECT b.user_id
    FROM stackoverflow.users AS u
    JOIN stackoverflow.badges AS b ON u.id = b.user_id
    GROUP BY b.user_id
    HAVING COUNT(name) > 1000)
AND title IS NOT NULL

''')

,title
0,What's the strangest corner case you've seen in C# or .NET?
1,What's the hardest or most misunderstood aspect of LINQ?
2,What are the correct version numbers for C#?
3,Project management to go with GitHub


4 поста написаны пользователями, имеющих более  1 тыс. значков.

### Выгрузим данные о пользователях из Канады (англ. Canada) и разделим пользователей на три группы в зависимости от количества просмотров их профилей:
- пользователям с числом просмотров больше либо равным 350 присвоим группу 1;
- пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
- пользователям с числом просмотров меньше 100 — группу 3.

Отобразим в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с количеством просмотров меньше либо равным нулю исключим из итоговой таблицы.

In [20]:
get_sql_data(
'''
SELECT 
    id,
    views,
    CASE
        WHEN views >= 350 THEN 1
        WHEN views >= 100 AND views < 350 THEN 2
        WHEN views < 100 THEN 3
     END
FROM stackoverflow.users
WHERE location LIKE '%Canada%' AND views != 0

''')

,id,views,case
0,22,1079,1
1,34,1707,1
2,37,757,1
3,41,174,2
4,42,1063,1
...,...,...,...
832,431315,103,2
833,266220,13,3
834,474548,24,3
835,455635,17,3


Из Канады, согласно условиям выгрузки, 837 пользователей, это около 3.5% от всех пользователей.

### Дополним предыдущий запрос. Отобразим лидеров каждой группы  — 
пользователей, которые набрали максимальное число просмотров в своей группе. Выведем поля с идентификатором пользователя, группой и количеством просмотров. Отсортируем таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [21]:
get_sql_data(
'''
WITH i AS 
    (SELECT 
        id,
        views,
        CASE
            WHEN views >= 350 THEN 1
            WHEN views >= 100 and views < 350 THEN 2
            WHEN views < 100 THEN 3
         END AS rang
    FROM stackoverflow.users
    WHERE location LIKE '%Canada%' AND views != 0),
j AS
    (SELECT *,
        MAX(views) OVER (PARTITION BY rang) AS max_views
    FROM i)
    
SELECT
    id,
    rang,
    views
FROM j
WHERE views = max_views
ORDER BY views DESC, id

''')

,id,rang,views
0,3153,1,21991
1,46981,2,349
2,3444,3,99
3,22273,3,99
4,190298,3,99


Из Канады в 1-й и 2-й группах по одному лидеру, в 3-й группе 3 пользователя с максимальным числом просмотров. Максимальное количество просмотров у пользователя 3153.

### Посчитаем ежедневный прирост новых пользователей в ноябре 2008 года. Сформируем таблицу с полями:
- номер дня;
- число пользователей, зарегистрированных в этот день;
- сумму пользователей с накоплением.

In [22]:
get_sql_data(
'''
SELECT  
    EXTRACT(day FROM CAST(creation_date AS timestamp)) AS day,
    COUNT(DISTINCT id) AS users_in_day,
    SUM(COUNT(DISTINCT id)) OVER (ORDER BY EXTRACT(day FROM CAST(creation_date AS timestamp)))
FROM stackoverflow.users
WHERE CAST(DATE_TRUNC('month', creation_date) AS date) = '2008-11-01'
GROUP BY day
ORDER BY day

''')

,day,users_in_day,sum
0,1.0,34,34.0
1,2.0,48,82.0
2,3.0,75,157.0
3,4.0,192,349.0
4,5.0,122,471.0
5,6.0,132,603.0
6,7.0,104,707.0
7,8.0,42,749.0
8,9.0,45,794.0
9,10.0,93,887.0


Минимальное количество пользователей в ноябре 2008 года регистрировались 1, 15 и 30 числа (32-34 новых пользователей). Больше всего пользователей зарегистрировано 4 ноября (192).

### Для каждого пользователя, который написал хотя бы один пост, найдем интервал между регистрацией и временем создания первого поста. 
Отобразим:
- идентификатор пользователя;
- разницу во времени между регистрацией и первым постом.

Для выполнения запроса используем данные из таблиц о пользователях и постах.

In [23]:
get_sql_data(
'''
SELECT
    DISTINCT p.user_id,
    (MIN(p.creation_date) OVER (PARTITION BY p.user_id) - u.creation_date) AS time_dt
FROM stackoverflow.posts AS p
JOIN stackoverflow.users AS u ON p.user_id = u.id

''')

,user_id,time_dt
0,27088,22 days 10:32:25
1,4666,4 days 13:51:01
2,43473,0 days 00:00:00
3,761503,0 days 00:00:00
4,9293,8 days 00:39:24
...,...,...
18844,34508,34 days 19:26:31
18845,18334,6 days 14:36:26
18846,4014,42 days 04:46:18
18847,1658,1 days 14:20:32


В столбце *time_dt* отразили интервал между датой регистрации и датой создания первого поста. Всего публиковали посты 18849 пользователей из общего количества пользователей (24044), что составляет 78.39%.

### Выведем общую сумму просмотров у постов, опубликованных в каждый месяц 2008 года. 
Если данных за какой-либо месяц в базе нет, такой месяц пропустим. Результат отсортируем по убыванию общего количества просмотров.

In [24]:
get_sql_data(
'''
SELECT
    CAST(DATE_TRUNC('month', creation_date) AS date) AS month,
    SUM(views_count) AS sum_views
FROM stackoverflow.posts
WHERE EXTRACT(year FROM creation_date) = '2008'
GROUP BY month
ORDER BY sum_views DESC

''')

,month,sum_views
0,2008-09-01,452928568
1,2008-10-01,365400138
2,2008-11-01,221759651
3,2008-12-01,197792841
4,2008-08-01,131367083
5,2008-07-01,669895


Больше всего просмотров у постов, опубликованных в сентябре 2008 года, менее всего просмотров у июльских постов.

### Выведем имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. 
Вопросы, которые задавали пользователи, не будем учитывать. Для каждого имени пользователя выведем количество уникальных значений user_id. Отсортируем результат по полю с именами в лексикографическом порядке.

Для выполнения запроса понадобятся данные из 3 таблиц:
- о постах;
- о типах постов;
- о пользователях.

In [25]:
get_sql_data(
'''
SELECT 
    u.display_name,
    COUNT(DISTINCT user_id)
FROM stackoverflow.posts AS p
JOIN stackoverflow.post_types AS pt ON p.post_type_id=pt.id
JOIN stackoverflow.users AS u ON p.user_id = u.id
WHERE pt.type = 'Answer' and 
    (DATE_TRUNC('day', p.creation_date) >= DATE_TRUNC('day', u.creation_date)
    AND DATE_TRUNC('day', p.creation_date) <= DATE_TRUNC('day', u.creation_date) + INTERVAL '1 month')
GROUP BY u.display_name
HAVING count(type) > 100
ORDER BY u.display_name

''')

,display_name,count
0,1800 INFORMATION,1
1,Adam Bellaire,1
2,Adam Davis,1
3,Adam Liss,1
4,Alan,8
...,...,...
74,lomaxx,1
75,mattlant,1
76,paxdiablo,1
77,tvanfosson,1


В полученной таблице 79 записей. Видим пользователей с неуникальными именами, например, с именем "Alan" 8 пользователей.

### Выведем количество постов за 2008 год по месяцам и отберем посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. 
Отсортируем таблицу по значению месяца по убыванию.

Для выполнения запроса нужны данные из таблиц о пользователях и о постах.

In [26]:
get_sql_data(
'''
WITH i AS
    (SELECT DISTINCT u.id
    FROM stackoverflow.users AS u
    JOIN stackoverflow.posts AS p ON u.id = p.user_id
    WHERE CAST(DATE_TRUNC('month', u.creation_date) AS date) = '2008-09-01' AND
        CAST(date_trunc('month', p.creation_date) AS date) = '2008-12-01')
        
SELECT
    CAST(DATE_TRUNC('month', creation_date) AS date) AS month,
    COUNT(id)
FROM stackoverflow.posts
WHERE user_id in (SELECT DISTINCT u.id
    FROM stackoverflow.users AS u
    JOIN stackoverflow.posts AS p ON u.id = p.user_id
    WHERE CAST(DATE_TRUNC('month', u.creation_date) AS date) = '2008-09-01' AND
        CAST(DATE_TRUNC('month', p.creation_date) AS date) = '2008-12-01')
GROUP BY month
ORDER BY month DESC

''')

,month,count
0,2008-12-01,17641
1,2008-11-01,18294
2,2008-10-01,27171
3,2008-09-01,24870
4,2008-08-01,32


По таблице видим  32 поста в августе от пользователей, зарегистрированных в сентябре. Значит, в данных имеются ошибки.

### Используя данные о постах, выведем несколько полей:
- идентификатор пользователя, который написал пост;
- дата создания поста;
- количество просмотров у текущего поста;
- сумма просмотров постов автора с накоплением.
Данные в таблице отсортируем по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [27]:
get_sql_data(
'''
SELECT
    user_id,
    creation_date,
    views_count,
    SUM(views_count) OVER (PARTITION BY user_id ORDER BY creation_date) AS sum_views
FROM stackoverflow.posts
ORDER BY user_id

''')

,user_id,creation_date,views_count,sum_views
0,1,2008-07-31 23:41:00,480476,480476
1,1,2008-07-31 23:55:38,136033,616509
2,1,2008-07-31 23:56:41,0,616509
3,1,2008-08-04 02:45:08,0,616509
4,1,2008-08-04 04:31:03,0,616509
...,...,...,...,...
243791,5696608,2008-12-23 16:00:37,0,2804
243792,5696608,2008-12-23 17:35:09,0,2804
243793,5696608,2008-12-24 01:02:48,0,2804
243794,5696608,2008-12-30 14:34:45,0,2804


Получили обновленную таблицу о постах каждого автора с накоплением просмотров.

### Определим, сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? 
Для каждого пользователя отберем дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число.

In [28]:
get_sql_data(
'''
WITH i AS 
    (SELECT user_id,
        COUNT(DISTINCT CAST(DATE_TRUNC('day', creation_date) AS date)) AS day_act
    FROM stackoverflow.posts
    WHERE CAST(DATE_TRUNC('day', creation_date) AS date) BETWEEN '2008-12-01' AND '2008-12-07'
    GROUP BY user_id)
    
    
SELECT ROUND(AVG(day_act))
FROM i

''')

,round
0,2.0


В среднем, с 1 по 7 декабря 2008 года, пользователи взаимодействовали с платформой 2 дня. 

### На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразим таблицу со следующими полями:
- Номер месяца.
- Количество постов за месяц.
- Процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.

Если постов стало меньше, значение процента будем отрицательным, если больше — положительным. Округлим значение процента до двух знаков после запятой.

In [29]:
get_sql_data(
'''
WITH i AS 
    (SELECT
        CAST(DATE_TRUNC('month', creation_date) AS date) AS month,
        COUNT(id) AS count_posts
    FROM stackoverflow.posts
    GROUP BY month
    ORDER BY month),

 j AS 
    (SELECT *,
        LAG(count_posts) OVER () AS pr_count_posts
      
    FROM i) 
    
SELECT EXTRACT(month FROM month),
    count_posts,
    ROUND(((count_posts - LAG(count_posts) OVER ())::numeric / LAG(count_posts) OVER ())*100, 2) AS percent
FROM i
WHERE month > '2008-08-01'

''')

,extract,count_posts,percent
0,9.0,70371,NaN
1,10.0,63102,-10.33
2,11.0,46975,-25.56
3,12.0,44592,-5.07


Мы наблюдаем ежемесячное снижение количества постов. Максимальное снижение произошло в ноябре (25.56%).

### Найдем пользователя, который опубликовал больше всего постов за всё время с момента регистрации. Выведем данные его активности за октябрь 2008 года в таком виде:
- номер недели;
- дата и время последнего поста, опубликованного на этой неделе.

Сначала выведем id этого пользователя и количество его постов:

In [30]:
get_sql_data(
'''
SELECT user_id,
       COUNT(id) AS max_posts
FROM stackoverflow.posts
GROUP BY user_id
ORDER BY max_posts DESC
LIMIT 1

''')

,user_id,max_posts
0,22656,1174


И теперь выведем активности этого пользователя за октябрь 2008:

In [31]:
get_sql_data(
'''
WITH i AS 
    (SELECT user_id,
        COUNT(id) AS max_posts
    FROM stackoverflow.posts
    GROUP BY user_id
    ORDER BY max_posts DESC
    LIMIT 1),

j AS 
    (  
    SELECT
        EXTRACT(week from creation_date) AS date_part,
        creation_date,
        MAX(creation_date) OVER (PARTITION BY EXTRACT(week FROM creation_date)) AS dt_last_posts

    FROM stackoverflow.posts
    WHERE user_id IN (select user_id FROM i) AND
        CAST(DATE_TRUNC('month', creation_date ) AS date) = '2008-10-01')
    
SELECT DISTINCT date_part,
    dt_last_posts
FROM j
ORDER BY date_part

''')

,date_part,dt_last_posts
0,40.0,2008-10-05 09:00:58
1,41.0,2008-10-12 21:22:23
2,42.0,2008-10-19 06:49:30
3,43.0,2008-10-26 21:44:36
4,44.0,2008-10-31 22:16:01


Максимальное количество постов 1174 у пользователя с id 22656, он демонстрирует регулярную еженедельную активность в течение октября 2008.

### Выводы

- Всего 1355 вопросов, которые набрали более 300 очков или не менее 100 раз были добавлены в "Закладки".
- 7047 уникальных пользователя получили значки сразу в день регистрации. Это 29.3% от общего количества пользователей.
- Максимальное количество 36 голосов типа Close поставил пользователь с id 20646.
- С 15 ноября по 15 декабря 2008 года больше всех значков (149) за установленный период получил пользователь с id 22656.
- 4 поста написаны пользователями, имеющих более  1 тыс. значков.
- Минимальное количество пользователей в ноябре 2008 года регистрировались 1, 15 и 30 числа (32-34 новых пользователей). Больше всего пользователей зарегистрировано 4 ноября (192).
- Всего публиковали посты 18849 пользователей из общего количества пользователей (24044), что составляет 78.39%. Максимальное количество постов 1174 у пользователя с id 22656.
- Больше всего просмотров у постов, опубликованных в сентябре 2008 года, менее всего просмотров у июльских постов.
- Не все имена пользователей являются уникальными.
- В данных имеются ошибки, т.к. присутствуют посты в августе 2008 от пользователей, зарегистрированных в сентябре 2008.
- В среднем, с 1 по 7 декабря 2008 года, пользователи взаимодействовали с платформой 2 дня. 
- с 1 сентября по 31 декабря 2008 года мы наблюдаем ежемесячное снижение количества постов от месяца к месяцу. Максимальное снижение произошло в ноябре (25.56%).